In [1]:
import torch.nn as nn
import torch
import math

batch_size = 1
image_size = 224
dim_in = 3
emb_dim = 16
c_group = 1
kernel_size = 3
stride = 2
c_group = 1

image = torch.zeros(batch_size, dim_in, image_size, image_size)
for b in range(batch_size):
    for c in range(dim_in):
        for i in range(image_size):
            for j in range(image_size):
                image[b, c, i, j] = (b+c+i+j) % 256

kernel = torch.zeros(emb_dim, dim_in, kernel_size, kernel_size)
kernel[:, :, :, :] = 0.1

c_group = math.gcd(dim_in, emb_dim) if c_group == 1 else c_group
padding = math.ceil(((kernel_size - 1) * 1 + 1 - stride) / 2)
print('padding: ', padding)
conv = nn.Conv2d(dim_in, emb_dim, kernel_size, stride, padding, dilation=1, groups=c_group, bias=False)
conv.weight.data = kernel
image = conv(image)
norm = nn.BatchNorm2d(emb_dim)
result = norm(image)

gamma = norm.weight
beta = norm.bias

#print(image.shape)
#print(image[0, 0, :, :])

print(gamma)
print(beta)
print(result[0, 0, :, :])

padding:  1
Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       requires_grad=True)
Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       requires_grad=True)
tensor([[-1.7490, -1.7293, -1.7110,  ...,  0.2403,  0.2585,  0.2767],
        [-1.7293, -1.6928, -1.6655,  ...,  1.2615,  1.2889,  1.3162],
        [-1.7110, -1.6655, -1.6381,  ...,  1.2889,  1.3162,  1.3436],
        ...,
        [ 0.2403,  1.2615,  1.2889,  ...,  0.7144,  0.7418,  0.7691],
        [ 0.2585,  1.2889,  1.3162,  ...,  0.7418,  0.7691,  0.7965],
        [ 0.2767,  1.3162,  1.3436,  ...,  0.7691,  0.7965,  0.8238]],
       grad_fn=<SliceBackward0>)
